## Abdulazeez EDA fil 

## Statsbidrag per utbildningsområde (med och utan moms)

In [44]:
import pandas as pd

df_statsbidrag = pd.read_csv("data/statsbidrag_schablonnivåer.csv", encoding="utf-8")

df_statsbidrag.columns = df_statsbidrag.columns.str.strip()

print("Kolumner i CSV-filen:\n", df_statsbidrag.columns.tolist())


Kolumner i CSV-filen:
 ['Utbildningsområde', 'Utan momskompensation', 'Med momskompensation']


In [ ]:
schablon_per_område = (
    df_statsbidrag.groupby("Utbildningsområde")[["Utan momskompensation", "Med momskompensation"]]
    .mean()
    .reset_index()
    .sort_values("Utbildningsområde")
)

schablon_per_område.index.name = "id"
schablon_per_område.reset_index(inplace=True)

print(schablon_per_område)


    id                             Utbildningsområde  Utan momskompensation  \
0    0                                       Data/IT                69900.0   
1    1       Ekonomi, administration och försäljning                62900.0   
2    2                      Friskvård och kroppsvård                74800.0   
3    3                 Hotell, restaurang och turism                64900.0   
4    4       Hälso- och sjukvård samt socialt arbete                67000.0   
5    5                  Journalistik och information                66600.0   
6    6                                       Juridik                60500.0   
7    7                      Kultur, media och design                84400.0   
8    8  Lantbruk, djurvård, trädgård, skog och fiske               110200.0   
9    9                    Pedagogik och undervisning                71300.0   
10  10                Samhällsbyggnad och byggteknik                70400.0   
11  11                             Säkerhetstjänster

In [35]:
import plotly.graph_objects as go

fig = go.Figure()
for col in ["Utan momskompensation", "Med momskompensation"]:
    fig.add_trace(go.Bar(
        x=schablon_per_område["Utbildningsområde"],
        y=schablon_per_område[col],
        name=col
    ))

fig.update_layout(
    title="Genomsnittligt schablonbelopp per utbildningsområde",
    barmode="group",
    xaxis_title="Utbildningsområde",
    yaxis_title="Schablonbelopp (kr)",
    xaxis_tickangle=-45
)
fig.show()


In [36]:
statsbidrag_total = pd.merge(
    beviljade,
    schablon_per_område,
    on="Utbildningsområde",
    how="left"
)

statsbidrag_total["Totalt statsbidrag utan moms"] = (
    statsbidrag_total["Totalt antal beviljade platser"] * statsbidrag_total["Utan momskompensation"]
)

statsbidrag_total["Totalt statsbidrag med moms"] = (
    statsbidrag_total["Totalt antal beviljade platser"] * statsbidrag_total["Med momskompensation"]
)

statsbidrag_total.head()


,id_x,Utbildningsområde,Totalt antal beviljade platser,Antal beviljade platser start 2024,Antal beviljade platser start och slut 2024,Antal beviljade platser start 2025,id_y,Utan momskompensation,Med momskompensation,Totalt statsbidrag utan moms,Totalt statsbidrag med moms
0,0,Data/IT,4159,1459,1276,1424,0,69900.0,74100.0,290714100.0,308181900.0
1,1,"Ekonomi, administration och försäljning",7148,2822,1714,2612,1,62900.0,66600.0,449609200.0,476056800.0
2,2,Friskvård och kroppsvård,0,0,0,0,2,74800.0,79200.0,0.0,0.0
3,3,"Hotell, restaurang och turism",586,132,322,132,3,64900.0,68700.0,38031400.0,40258200.0
4,4,Hälso- och sjukvård samt socialt arbete,3155,1140,940,1075,4,67000.0,71000.0,211385000.0,224005000.0


In [37]:
import plotly.express as px

fig = px.bar(
    statsbidrag_total,
    x="Utbildningsområde",
    y="Totalt statsbidrag med moms",
    title="Totalt statsbidrag per utbildningsområde (med moms)",
    labels={"Totalt statsbidrag med moms": "Statsbidrag (kr)"},
    color="Utbildningsområde"
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [38]:
total_utan_moms = statsbidrag_total["Totalt statsbidrag utan moms"].sum()
total_med_moms = statsbidrag_total["Totalt statsbidrag med moms"].sum()

print(f"Totalt statsbidrag utan moms: {total_utan_moms:,.0f} kr")
print(f"Totalt statsbidrag med moms: {total_med_moms:,.0f} kr")


Totalt statsbidrag utan moms: 1,677,555,100 kr
Totalt statsbidrag med moms: 1,777,230,800 kr


In [42]:
kpi_full = pd.merge(
    kpi,
    ansökta_platser_utbildningsområde[["Sökt utbildningsområde", "Total ansökta platser"]],
    left_on="Utbildningsområde",
    right_on="Sökt utbildningsområde",
    how="left"
)


kpi_full["Beviljandegrad (%)"] = (
    kpi_full["Beviljade platser"] / kpi_full["Total ansökta platser"] * 100
).round(1)


kpi_full.drop(columns=["Sökt utbildningsområde"], inplace=True)


kpi_full = kpi_full.sort_values("Totalt statsbidrag (med moms)", ascending=False)


kpi_full.reset_index(drop=True, inplace=True)
kpi_full


,Utbildningsområde,Beviljade platser,Schablonbelopp (med moms),Totalt statsbidrag (med moms),Statsbidrag per plats (kr),Total ansökta platser,Beviljandegrad (%)
0,"Ekonomi, administration och försäljning",7148,66600.0,476056800.0,66600.0,16316,43.8
1,Data/IT,4159,74100.0,308181900.0,74100.0,18458,22.5
2,Teknik och tillverkning,2986,91000.0,271726000.0,91000.0,9427,31.7
3,Samhällsbyggnad och byggteknik,3110,74600.0,232006000.0,74600.0,9645,32.2
4,Hälso- och sjukvård samt socialt arbete,3155,71000.0,224005000.0,71000.0,11992,26.3
5,"Lantbruk, djurvård, trädgård, skog och fiske",467,116700.0,54498900.0,116700.0,1875,24.9
6,"Hotell, restaurang och turism",586,68700.0,40258200.0,68700.0,2656,22.1
7,Pedagogik och undervisning,518,75500.0,39109000.0,75500.0,1659,31.2
8,Journalistik och information,482,70600.0,34029200.0,70600.0,1020,47.3
9,Juridik,518,64100.0,33203800.0,64100.0,844,61.4
